### 0. Notebook Setup

##### Importing Appropriate Packages

In [1]:
# General Packages

import numpy as np
import pickle
import os
import sys
import json


# Torch Specific

import torch
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader

from torchvision import models,transforms,datasets

from PIL import Image


# Project Specific

sys.path.append(str(os.getcwd() + '/scripts'))
from Preprocessing import MultiLabelDataset

###### Constants and Parameters

In [16]:
BATCH_SIZE = 16
IS_SHUFFLE = True
NUM_WORKERS = 0

LEARNING_RATE = 0.001

###### Machine Setup

In [17]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('Using gpu: %s ' % torch.cuda.is_available())

Using gpu: False 


### 1. Loading Data

##### Importing Data

In [18]:
# Loading the tupple of tensors from the pickle file
with open('data/train_data.pickle', 'rb') as handle:
    train = pickle.load(handle)

###### Data Loader

In [19]:
# Building the data loader from the tupple of tensors
train_loader = DataLoader(train,
                          batch_size=BATCH_SIZE,
                          shuffle=IS_SHUFFLE,
                          num_workers=NUM_WORKERS)

### 2. Building Model

##### Transfer Learning

In [54]:
# Loading the pretrained VGG model
model_vgg = models.vgg16(pretrained=True)

# Freezing all layers
for param in model_vgg.parameters():
    param.requires_grad = False

# Adding linear layer
model_vgg.classifier._modules['6'] = torch.nn.Linear(4096, 13)
model_vgg.classifier._modules['7'] = torch.nn.Sigmoid()

# Defining the optimizer
optimizer_vgg = torch.optim.SGD(model_vgg.classifier[6].parameters(),lr = LEARNING_RATE)

In [55]:
def train_model(model, dataloader, size, epochs=1, optimizer=None):
    model.train()
    
    for epoch in range(epochs):
        running_loss = 0.0
        running_corrects = 0
        for inputs, classes in train_loader:
            
            inputs_try , labels_try = inputs.to(device), classes.to(device)
            outputs_try = model(inputs_try)
            
            loss = torch.nn.BCELoss()
            loss = loss(outputs_try, labels_try)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            # statistics
            running_loss += loss.data.item()
        
        epoch_loss = running_loss / size
        epoch_acc = running_corrects.data.item() / size
        print('Loss: {:.4f} Acc: {:.4f}'.format(epoch_loss, epoch_acc))

In [56]:
train_model(model_vgg, 
            train_loader,
            size=len(train),
            epochs=2,
            optimizer=optimizer_vgg)

RuntimeError: size mismatch, m1: [16 x 4096], m2: [32 x 13] at ../aten/src/TH/generic/THTensorMath.cpp:197